# Download data

In [2]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Step 1: Website URL and folder to save CSV files
url = "https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=aecb88a7e2b73410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default"  # Replace with the actual URL
download_folder = "csv_files"

# Create the download folder if it doesn't exist
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Step 2: Fetch the webpage content
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to load page {url}")

# Step 3: Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Step 4: Find all the links to CSV files
csv_links = []
for link in soup.find_all("a", href=True):
    href = link["href"]
    if href.endswith(".csv"):  # Check if the link is a CSV file
        full_url = urljoin(url, href)  # Create an absolute URL
        csv_links.append(full_url)

# Step 5: Download all the CSV files
for csv_link in csv_links:
    csv_file_name = os.path.join(download_folder, os.path.basename(csv_link))
    
    # Download the CSV file
    with requests.get(csv_link, stream=True) as r:
        if r.status_code == 200:
            with open(csv_file_name, "wb") as file:
                for chunk in r.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"Downloaded: {csv_file_name}")
        else:
            print(f"Failed to download: {csv_link}")


# Merge CSV files

In [24]:
import os
import pandas as pd

# Step 1: Path to the folder containing CSV files
folder_path = "csv_files"  # Replace with the path to the folder where CSV files were downloaded

# Step 2: List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Step 3: Initialize an empty list to store DataFrames
dataframes = []

# Step 4: Read each CSV file and append to the list
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path, sep=';')  # Read the CSV file
    dataframes.append(df)  # Append the DataFrame to the list

# Step 5: Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)




# Read-in metadata

In [25]:
# Table with codes for measures and experimental techniques
df_exp=pd.read_csv('metadata/Magnitudes.csv', index_col=0)
# Table with codes for the stations
df_estaciones=pd.read_csv('metadata/Estaciones.csv', index_col=0)

In [26]:
# Turn both tables into dictionaries
dict_estaciones=df_estaciones.to_dict()['NOMBRE']
dict_magnitudes=df_exp.to_dict()['Magnitud']

In [27]:
# Make another one for the experimental techniques
df_exp.index=df_exp.Codigo_Tecnica
dict_tecnicas=df_exp.to_dict()['Tecnica_de_medida']

# Clean-up the table

In [28]:
# Remove validation columns
combined_df=combined_df[[i for i in combined_df.columns if 'V' not in i]].copy()
# Add Exp. Technique and Station columns
combined_df['Tecnica']=[int(i.split('_')[-1]) for i in combined_df.PUNTO_MUESTREO]
combined_df['Estacion']=[int(i.split('_')[0]) for i in combined_df.PUNTO_MUESTREO]
# Apply dictionaries
combined_df['Estacion']=combined_df['Estacion'].replace(dict_estaciones)
combined_df['Medida']=combined_df['MAGNITUD'].replace(dict_magnitudes)
combined_df['Tecnica']=combined_df['Tecnica'].replace(dict_tecnicas)
combined_df.Tecnica=combined_df.Tecnica.astype(str)

In [29]:
# Restrict to the relevant columns
sel_cols=list(['Estacion','Medida','Tecnica','ANO','MES'])+list([i for i in combined_df.columns if i[0]=='D'])
combined_df=combined_df[sel_cols].copy()

# Melt table, format columns and write out

In [30]:
# Melt all the columns with values per day
df_long=combined_df.melt(id_vars=['Estacion','Medida','Tecnica','ANO','MES'])
# Rename columns
df_long.columns=['Estacion','Medida','Tecnica','Year','Month','Day','Value']
# Turn days into integers
df_long['Day']=[int(i[1:]) for i in df_long['Day']]
# Make data column
df_long['Date']=df_long['Day'].astype(str)+'-'+df_long['Month'].astype(str)+'-'+df_long['Year'].astype(str)
df_long['Date']=pd.to_datetime(df_long['Date'], format='%d-%m-%Y', errors='coerce')
# There are invalid dates included, remove them
df_long=df_long[~df_long['Date'].isna()].copy()
# Remove entries with values of 0
df_long=df_long[df_long.Value!=0].copy()

In [31]:
# Write out to a parquet file
df_long.to_parquet('data/AirData_raw.parquet')